In [11]:
import pathlib
from tkinter import filedialog
import tkinter
import glob
import os
tkinter.Tk().withdraw()
#path1=os.path.join("D:\\","Users","Abu Bakr","Desktop","Raw images")
path1=filedialog.askdirectory(title='path for raw images')
img_list=[x for x in sorted(glob.glob(path1+"/*/*.jpg"))]

In [13]:
img_list


['/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A006/A006, acinar, 20x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A006/A006, acinar, 5x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A008/A008, paillary, 20x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A008/A008, paillary, 5x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A012/A012, acinar, 20x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A012/A012, acinar, 5x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A015/A015, acinar, 20x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A015/A015, acinar, 5x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A016/A016, solid, 20x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A016/A016, solid, 5x.jpg',
 '/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/H

In [14]:

#os.environ["CUDA_VISIBLE_DEVICES"]="2"
import cv2
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
from cuml.cluster import KMeans
import shutil

from cucim.skimage.transform import warp
one_seg_only=True
one_seg_rem=False
seg_img=False
from cucim.skimage import registration
from skimage import io

size='3328'
orient='Horizontal'
layers=['DAPI','CD3']



#path2=filedialog.askdirectory(title='path for processed images')



In [15]:
from math import sqrt




def closest_color(centers):
    print(centers)
    closest_color_ctrs=[]
    i=0
    for rgb in centers:
        r, g, b = rgb
        color_diffs_pctr = []
        
            #compensate for float
        color_diff = sqrt(abs(r)**2 + abs(g)**2 + abs(b)**2)
        
        closest_color_ctrs.append((color_diff,i))
        i=i+1
    print(max(closest_color_ctrs)[0])
        #if(min(closest_color_ctrs)[0]<200):
    return max(closest_color_ctrs)[1]
      #  else:
         #   return -1

In [17]:
img_store=0
for img_no in img_list:
    print(img_no)
    #if(os.path.exists(os.path.join(path2,f"{img_store:06d}"))==False):
     #   os.makedirs(os.path.join(path2,f"{img_store:06d}"),exist_ok=False)
    img=cv2.imread(img_no)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    pixel_values = img.reshape((-1, 3))
    pixel_values = cp.float32(pixel_values)
    
    
    
    k=4
    #print("input:")
    #print(b)

    print("Calling fit")
    kmeans_float = KMeans(n_clusters=k,max_iter=30,tol=0.000001)
    kmeans_float.fit(pixel_values)

    labels=kmeans_float.labels_
    centers=kmeans_float.cluster_centers_
    
    
    
    #centers = np.uint8(centers)
    blue_idx=closest_color(centers)
    
    
    # flatten the labels array --->Check
    labels = labels.flatten()

    segmented_image = centers[labels.flatten()]
    segmented_image = segmented_image.reshape(img.shape)
    #cv2.imwrite(os.path.join(path2,f"{img_store:06d}","segmented_image "+f"{img_store:06d}"+".png"),segmented_image)
 
    for i in range (k):
        if(seg_img==True):
            temp = cp.copy(segmented_image) # segmented_image flattens the image, ie processes the image and brings down the number of 
                                # colors to k colors and changes each cluster's color to the centroid
        else:
            temp=cp.copy(img) #for real image
    # convert to the shape of a vector of pixel values
        temp = temp.reshape((-1, 3))
    # color (i.e cluster) to disable
        cluster = i
        #temp[labels == cluster] = [0, 0, 0] #for removing color belonging to a cluster

        temp[labels == cluster] = [0, 0, 0] # for removing any color not belonging to cluster


# convert back to original shape
        temp = temp.reshape(img.shape)
        t2=cv2.cvtColor(temp,cv2.COLOR_RGB2BGR)
           # t2=cv2.medianBlur(t2,25)
        if(i==blue_idx):
            a=pathlib.Path(img_no)
            cv2.imwrite(str(a.parent)+'/'+os.path.split(img_no)[-1].split(".")[0]+'_w.jpg',t2)


    img_store=img_store+1

/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A006/A006, acinar, 20x.jpg
Calling fit
[[214.82047  195.67256  217.52158 ]
 [147.26753  104.95757  166.90004 ]
 [ 12.269657  10.660787  12.841734]
 [192.23271  145.35548  193.67807 ]]
362.97550825858457
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A006/A006, acinar, 5x.jpg
Calling fit
[[206.2242  163.41522 202.4771 ]
 [134.87196 100.3627  156.19327]
 [224.4369  209.48723 224.59067]
 [179.68262 134.23627 182.50778]]
380.3916286846159
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A008/A008, paillary, 20x.jpg
Calling fit
[[220.91354  214.99834  223.28053 ]
 [159.50127  112.00702  172.70435 ]
 [ 16.307243  13.668936  16.766165]
 [197.28893  152.33743  196.8565  ]]
380.63272975122453
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A008/A008, paillary, 5x.jpg
Calling fit
[[203.59135 160.53755 202.50957]
 [135.41365  94.85466 152.1927 ]
 [227.2507  215.10014 227.50159]
 [177.224   

[[227.87718 217.97784 232.24089]
 [136.88683 107.38211 169.53737]
 [215.69589 177.17575 213.24826]
 [189.57324 144.76868 194.70996]]
391.63526638919893
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A071/A071, acinar, 5x.jpg
Calling fit
[[234.51382 224.36644 236.33101]
 [189.01596 149.35455 190.20277]
 [216.17455 183.51625 214.0505 ]
 [130.25311 102.12661 156.73645]]
401.48397098062225
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A072/A072, lepidic, 20x.jpg
Calling fit
[[149.45819  104.49451  175.47527 ]
 [ 10.762397  10.321797  10.99845 ]
 [230.68233  228.35632  234.31909 ]
 [212.09587  162.37996  209.6757  ]]
400.3328405965971
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A072/A072, lepidic, 5x.jpg
Calling fit
[[219.30544 182.0612  216.91037]
 [231.55942 232.41069 234.51569]
 [128.82483  94.0648  160.2469 ]
 [189.84007 137.6588  192.32567]]
403.2767038929515
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A073/A073, ac

[[206.68352  159.2572   207.68465 ]
 [220.1042   188.9173   219.92053 ]
 [117.77214   88.377815 163.23874 ]
 [171.24301  125.41683  188.62    ]]
364.00638125587517
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A122/A122, acinar, 5x.jpg
Calling fit
[[170.62372 126.68518 182.1279 ]
 [206.59102 162.9057  206.74052]
 [111.96424  82.52111 147.84615]
 [226.97453 191.84943 226.09694]]
373.42129946185827
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A124/A124, acniar, 20x.jpg
Calling fit
[[179.04819 130.52594 190.63318]
 [128.28088  89.9156  162.13605]
 [223.80585 195.07732 224.64821]
 [206.6355  159.96548 209.06494]]
372.3050284250271
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A124/A124, acniar, 5x.jpg
Calling fit
[[234.7745  214.65802 234.27708]
 [181.50435 137.7089  187.0621 ]
 [120.37838  87.27202 146.50598]
 [214.25832 173.6068  212.98009]]
395.07326699916655
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A126/A126, aci

[[202.697    148.36261  202.8981  ]
 [ 12.977994  10.80348   15.737462]
 [151.26167  100.539635 174.94252 ]
 [221.40125  193.06616  221.90166 ]]
368.148610912501
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A152/A152, acinar, 5x.jpg
Calling fit
[[182.7835  131.95831 186.08696]
 [231.5218  205.21034 231.87805]
 [127.80474  88.44003 153.55145]
 [209.8187  165.11128 209.23718]]
386.6278076136929
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A153/A153, acinar, 20x.jpg
Calling fit
[[214.88573 172.8807  213.62555]
 [133.41415 100.96373 169.37105]
 [225.92159 213.9545  228.35278]
 [187.88849 139.4328  196.11436]]
385.95606426378686
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A153/A153, acinar, 5x.jpg
Calling fit
[[210.6693   169.23596  209.47868 ]
 [232.3802   207.21457  232.35548 ]
 [177.7451   134.46436  184.24443 ]
 [117.835976  90.15909  147.3786  ]]
388.49389628925627
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A155

[[211.73906  170.82268  209.60857 ]
 [ 12.033809  10.675647  11.988379]
 [224.9892   199.54007  223.47241 ]
 [163.4064   133.31783  186.97813 ]]
374.6682492666266
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A184/A184, acinar, 5x.jpg
Calling fit
[[191.49724 150.67677 190.63739]
 [234.16687 211.11589 232.55376]
 [218.69414 183.53282 214.968  ]
 [117.12187 108.95809 160.48898]]
391.77198986790427
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A184/A184, solid nests of ADCA, 20x.jpg
Calling fit
[[206.4625   162.28436  205.41211 ]
 [ 74.256325  58.932278  72.806114]
 [223.4904   191.16147  220.31511 ]
 [158.74529  121.80552  178.53862 ]]
367.4634870183093
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A184/A184, solid nests of ADCA, 5x.jpg
Calling fit
[[212.46783  171.16652  208.0586  ]
 [230.3135   198.2332   226.73212 ]
 [122.946465 100.96161  152.794   ]
 [178.2808   137.94463  181.47722 ]]
379.1413534523917
/home/saab/Desktop/Raw_Data/I

...

[[197.18425  137.61652  197.84465 ]
 [ 10.1532     9.603358  10.490556]
 [151.90398  103.43488  172.96706 ]
 [213.74919  178.40178  216.97188 ]]
352.97692198312836
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A285/A285, acinar, 5x.jpg
Calling fit
[[217.72928  175.07623  218.34212 ]
 [174.54805  119.01578  181.51418 ]
 [205.87602  139.30426  199.73364 ]
 [127.475555  88.26856  154.53925 ]]
354.58568254433885
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A295/A295, papillary, 20x.jpg
Calling fit
[[225.39099  207.22061  225.56451 ]
 [155.04213  105.43904  172.56429 ]
 [ 11.534073  10.412573  12.130481]
 [200.13101  149.11638  201.51964 ]]
380.2904558725344
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A295/A295, papillary, 5x.jpg
Calling fit
[[229.84175 211.2366  228.63937]
 [208.66785 162.43814 207.38277]
 [131.10858  90.15418 152.95062]
 [178.64268 126.67589 181.95734]]
386.941980335119
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_

[[234.8333  224.99791 237.55614]
 [186.26187 139.40297 191.32014]
 [218.17995 178.10043 218.26967]
 [130.27995  97.86032 157.22899]]
402.7451493056886
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A377/A377, acinar, 20x.jpg
Calling fit
[[221.61606   184.12286   223.38815  ]
 [155.08157   119.14983   185.66331  ]
 [ 12.975026   11.1758585  13.036421 ]
 [195.40543   153.90614   208.179    ]]
364.5780756105529
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A377/A377, acinar, 5x.jpg
Calling fit
[[128.80981  103.400894 162.64568 ]
 [230.04152  196.29622  231.65564 ]
 [175.4946   137.47264  190.38501 ]
 [207.52452  168.68317  213.32822 ]]
380.9404697116828
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A379/A379, acinar, 20x.jpg
Calling fit
[[213.46701  161.15344  214.23291 ]
 [168.86148  121.97853  189.1315  ]
 [ 11.622492  10.343189  13.145723]
 [231.96785  215.0853   233.9572  ]]
393.4548756146994
/home/saab/Desktop/Raw_Data/Images_Raw_Data

[[128.3691    92.076515 158.99474 ]
 [225.84337  166.48866  215.73932 ]
 [231.63402  218.96722  232.01595 ]
 [192.32303  128.25783  192.11931 ]]
394.2491121667007
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A486/A486, acini and solid nests, 5x.jpg
Calling fit
[[227.85826   199.93295   227.90677  ]
 [163.70123   128.6474    182.39403  ]
 [ 15.2679205  12.556177   16.131165 ]
 [209.23128   169.49135   209.72246  ]]
379.25462168222225
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A486/A486, acini, 20x.jpg
Calling fit
[[210.1864   165.96753  210.92198 ]
 [167.05708  130.1172   188.99937 ]
 [ 10.687271   9.938711  10.749607]
 [224.14423  197.63834  224.33406 ]]
373.6673879097234
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A486/A486, solid nests, 20x.jpg
Calling fit
[[221.29552  193.80557  222.85254 ]
 [167.05417  125.83808  187.57648 ]
 [ 10.318588   9.629806  10.687203]
 [208.72044  162.77017  209.9152  ]]
369.0468262056662
/home/saab/

[[250.81268     3.2527957   2.0368867]
 [167.88283   104.323105  161.44185  ]
 [225.70421   184.33719   221.31212  ]
 [111.08637    53.26583   112.660645 ]]
365.9257337651378
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A504/A504, papillary, 20x.jpg
Calling fit
[[223.90143   195.64825   223.172    ]
 [162.70776   107.37943   175.63318  ]
 [  9.482815    8.984179    9.9994545]
 [203.31781   144.05415   200.72562  ]]
371.77389537745125
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A504/A504, papillary, 5x.jpg
Calling fit
[[231.75027 197.69705 228.9775 ]
 [181.84561 122.14055 182.17885]
 [210.23662 153.54108 205.27992]
 [138.80864  91.13161 155.23729]]
381.0813648107403
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A505/A505, acinar, 20x.jpg
Calling fit
[[158.9118   116.86216  176.60274 ]
 [195.85687  152.33173  198.54962 ]
 [ 24.44415   19.189533  23.555408]
 [217.85616  188.82353  216.59247 ]]
360.5938546728597
/home/saab/Desktop/Raw_D

[[189.91237  139.53339  198.4912  ]
 [215.75087  172.36182  215.49207 ]
 [151.0816   104.01243  172.99734 ]
 [ 54.646927 125.09185  221.09392 ]]
350.2768388236901
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A532/A532, acinar, 5x.jpg
Calling fit
[[228.81424  191.66031  228.23643 ]
 [173.18822  126.32551  182.83122 ]
 [128.40056   91.184616 153.48743 ]
 [204.46696  157.40427  206.30418 ]]
375.7412679161376
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A534/A534, papillary, 20x.jpg
Calling fit
[[225.26357  215.25255  228.76337 ]
 [157.57909  111.92506  181.03856 ]
 [  9.794261   9.135355  10.561993]
 [208.38872  158.47136  209.25081 ]]
386.5359119188193
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_sg/onco_sg/HE/A534/A534, papillary, 5x.jpg
Calling fit
[[230.63766 216.24289 232.96823]
 [128.23965  89.52119 153.5619 ]
 [214.54396 170.85126 213.71011]
 [182.20041 130.38518 186.71658]]
392.71989558586966
/home/saab/Desktop/Raw_Data/Images_Raw_Data/Onco_

In [9]:
img_list

[]

img_list_blue=[os.path.join(path2,f"{img_store:03d}","blue_nuclei"+".png" ) for img_store in range(len(img_list))]


img_list_dapi=[x for x in sorted(glob.glob(os.path.join(path1,"*",'dapi','*.png')) )]
img_list_cd3=[x for x in sorted(glob.glob(os.path.join(path1,"*",'cd3','*.png') ))]
#img_list_dapi=[x for x in sorted(glob.glob(os.path.join(path1,'dapi','tiles','*.png')) )]
#img_list_cd3=[x for x in sorted(glob.glob(os.path.join(path1,'cd','tiles','*.png') ))]
#img_list_cd45=[x for x in sorted(glob.glob(os.path.join(path1,'cd45','*.png')) )]

img_list_dapi

img_store=0
for img1,img2 in zip(img_list_blue,img_list_dapi):
    path3=os.path.join(path2,f"{img_store:03d}",'align')
    if(os.path.exists(os.path.join(path3))==False):
        os.makedirs(os.path.join(path3),exist_ok=False)
    img1=cv2.imread(img1,cv2.IMREAD_GRAYSCALE)
    img2=cv2.imread(img2,cv2.IMREAD_GRAYSCALE)
    img1=cv2.resize(img1,(3072,3072))
    img2=cv2.resize(img2,(3072,3072))
    rh1,img1_1 = cv2.threshold(img1,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    rh2,img2_1 = cv2.threshold(img2,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    #img1_1=cv2.bitwise_not(img1_1)
    #img2_1=cv2.bitwise_not(img2_1)
    cv2.imwrite(os.path.join(path3,"img1 of"+f"{img_store:03d}"+".png"),img1_1)
    cv2.imwrite(os.path.join(path3,"img2 of"+f"{img_store:03d}"+".png"),img2_1)
    
    print("thresholding done for "+f"{img_store:03d}")
# --- Compute the optical flow
    #v, u = registration.optical_flow_tvl1(img1_1, img2_1,attachment=5, tightness=0.3, num_warp=5, num_iter=10, tol=0.0001)
    v, u = registration.optical_flow_tvl1(cp.asarray(img1_1), cp.asarray(img2_1))
    print("alignment done for "+f"{img_store:03d}")
# --- Use the estimated optical flow for registration

    nr, nc = img1.shape
    row_coords, col_coords = cp.meshgrid(cp.arange(nr), cp.arange(nc),indexing='ij')
    image1_warp = warp(cp.asarray(img2_1), cp.array([row_coords + v, col_coords + u]),mode='nearest')
    img1_rgb=cv2.cvtColor(img1_1,cv2.COLOR_GRAY2BGR)
    image1_warp=image1_warp.astype(np.float32)
    img2_rgb=cv2.cvtColor(cp.asnumpy(image1_warp),cv2.COLOR_GRAY2BGR)
    red = np.zeros((3072,3072,3),dtype=cp.float32)
    red[:]=np.asarray((255,0,0))
    blue = np.zeros((3072,3072,3),dtype=cp.float32)
    blue[:]=np.asarray((0,0,255))
    plt.imsave(os.path.join(path3,"overlap of"+f"{img_store:03d}"+".png"),cv2.bitwise_not(img1_rgb)*blue+
               cv2.bitwise_not(img2_rgb)*red)
    plt.imsave(os.path.join(path3,"warp of"+f"{img_store:03d}"+".png"),img2_rgb,cmap='gray')
    img_to_process=[]
    img_to_process.append(img_list_dapi[img_store])
    img_to_process.append(img_list_cd3[img_store])
    #img_to_process.append(img_list_cd45[img_store])
    processed_imgs=[]
    for img2_main in img_to_process:
        img2_main=cv2.imread(img2_main)
        img2_main=cv2.resize(img2_main,(3072,3072))
        img2_r,img2_g,img2_b=cv2.split(img2_main)
        
        image1_r_warp = warp(cp.asarray(img2_r), cp.array([row_coords + v, col_coords + u]),
                       mode='nearest')
        plt.imsave(os.path.join(path3,"image_r of"+f"{img_store:03d}"+".png"),cp.asnumpy(image1_r_warp),cmap='gray')
        image1_g_warp = warp(cp.asarray(img2_g), cp.array([row_coords + v, col_coords + u]),
                       mode='nearest')
        plt.imsave(os.path.join(path3,"image_g of"+f"{img_store:03d}"+".png"),cp.asnumpy(image1_g_warp),cmap='gray')
        image1_b_warp = warp(cp.asarray(img2_b), cp.array([row_coords + v, col_coords + u]),
                       mode='nearest')
        plt.imsave(os.path.join(path3,"image_b of"+f"{img_store:03d}"+".png"),cp.asnumpy(image1_b_warp),cmap='gray')
        img1=cv2.imread(os.path.join(path3,"image_r of"+f"{img_store:03d}"+".png"))
        img2=cv2.imread(os.path.join(path3,"image_g of"+f"{img_store:03d}"+".png"))
        img3=cv2.imread(os.path.join(path3,"image_b of"+f"{img_store:03d}"+".png"))
        img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        img3=cv2.cvtColor(img3,cv2.COLOR_BGR2GRAY)
        np.multiply(img1, 1, out=img1, casting="unsafe")
        np.multiply(img2, 1, out=img2, casting="unsafe")
        np.multiply(img3, 1, out=img3, casting="unsafe")
        after = cv2.merge([img1,img2,img3])
        processed_imgs.append(after)
    for img_present,toi in zip(processed_imgs,layers):
                cv2.imwrite(os.path.join(path3,toi+f"{img_store:03d}"+".png"),img_present)
    img_store=img_store+1